In [1]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

In [3]:
!python -m atari_py.import_roms .\ROMS\ROMS

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\atari_py\__init__.py", line 1, in <module>
    from .ale_python_interface import *
  File "C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\atari_py\ale_python_interface.py", line 17, in <module>
    ale_lib = cdll.LoadLibrary(os.path.join(os.path.dirname(__file__),
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\ctypes\__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\ctypes\__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: Could not fi

# 1. Import dependencies

In [2]:
import gym 
import os
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [3]:
environment_name = "FrozenLake-v1"

In [4]:
env = gym.make(environment_name, render_mode="human")

In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        result = env.step(action)
        if isinstance(result, tuple) and len(result) >= 4:
            n_state, reward, done, info = result[:4]
            score += reward
        else:
            raise ValueError("The environment's step function does not return the expected number of values.")
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:0.0
Episode:2 Score:0.0
Episode:3 Score:0.0
Episode:4 Score:0.0
Episode:5 Score:0.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [6]:
# 0 NOOP 1 FIRE 2 UP 3 RIGHT 4 LEFT 5 RIGHTFIRE 6 LEFTFIRE
env.action_space.sample()

0

In [7]:
# obs_type="rgb" -> observation_space=Box(0, 255, (210, 160, 3), np.uint8)
env.observation_space.sample()

0

# 3. Train an RL Model

In [8]:
# env = gym.make(environment_name, render_mode="rgb_array")
# model = PPO(MlpPolicy, env, verbose=1)
# model.learn(total_timesteps=10000)
env = gym.make(environment_name, render_mode="rgb_array")
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [9]:
model.learn(total_timesteps=30000)

C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


-----------------------------
| time/              |      |
|    fps             | 2658 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1345       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01096439 |
|    clip_fraction        | 0.0204     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.38      |
|    explained_variance   | -9.51      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0107     |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.00411   |
|    value_loss           | 0.0173     |
----------------------------------------
-----------------------------------------
| time/   

# 4. Save and Reload Model

In [10]:
import os
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [11]:
model.save(PPO_path)

In [12]:
del model

In [13]:
model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [14]:
from stable_baselines3.common.evaluation import evaluate_policy

In [15]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(0.2, 0.4000000000000001)

In [16]:
env.close()

# 5. Test Model

In [17]:
env = gym.make(environment_name, render_mode="human")
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done: 
        print('info', info)
        break

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
env.close()

# 6. Viewing Logs in Tensorboard

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()